### CONFIGURATION

In [5]:
import pandas as pd
import numpy as np
import copy
import sys
import matplotlib.pyplot as plt
%matplotlib inline

In [6]:
# Hartwig imports
sys.path.append("/Users/peterpriestley/hmf/repos/scripts/analysisscripts") 
import analyseVCF as aVCF
import venn as vn



In [7]:

# BED FILE
BED_PATH = "/Users/peterpriestley/hmf/analyses/70-30sample/truthset/"
BED_FILE_NAME = "union13callableMQonlymerged_addcert_nouncert_excludesimplerep_excludesegdups_excludedecoy_excludeRepSeqSTRs_noCNVs_v2.19_2mindatasets_5minYesNoRatio.bed"
# TRUTH SET
SAMPLE_NAMES_TRUTH = {'NA12878':'70-30truth'}
VCF_PATH_TRUTH = "/Users/peterpriestley/hmf/analyses/70-30sample/truthset/"
VCF_FILE_NAME_TRUTH = "na12878-na24385-somatic-truth.vcf"
# COMBINED VCF CONFIG
VCF_PATH = "/Users/peterpriestley/hmf/analyses/70-30sample/160524/"
VCF_FILE_NAME = "CPCT11111111R_CPCT11111111T_merged_somatics.vcf"
SAMPLE_NAMES = {'CPCT11111111T.mutect':'mutect', \
               'CPCT11111111T.freebayes':'freebayes', \
               'TUMOR.strelka':'strelka', \
               'TUMOR.varscan':'varscan'}
#MELT VCF CONFIG
#VCF_PATH = "/Users/peterpriestley/hmf/analyses/70-30sample/160524/"
#VCF_FILE_NAME = "CPCT11111111R_CPCT11111111T_merged_somatics_snpEff_dbSNP_Cosmicv76_melted.vcf" 
#SAMPLE_NAMES = {'/sample/output/GIAB_SOMATIC-MIX/somaticVariants/CPCT11111111R_CPCT11111111T/CPCT11111111R_CPCT11111111T_merged_somatics_snpEff_dbSNP_Cosmicv76' \
#                    :'melted'}

<h3> Load VCFs and Prepare DF

In [8]:
## LOAD BED
bed = aVCF.loadBEDFile(BED_PATH,BED_FILE_NAME)
bed.reverse()
truthBed = copy.deepcopy(bed)

reading BED file. . .





In [ ]:
## LOAD TRUTH SET VCF
dfTruth = aVCF.loadVaraintsFromVCF(VCF_PATH_TRUTH,VCF_FILE_NAME_TRUTH, \
                                   SAMPLE_NAMES_TRUTH,True,truthBed)
dfTruth['chromPos']=dfTruth['chrom']+':'+dfTruth['pos']

reading vcf file. . .

reading VCF File line: 100000
reading VCF File line: 200000
reading VCF File line: 300000
reading VCF File line: 400000
reading VCF File line: 500000
reading VCF File line: 600000
reading VCF File line: 700000
reading VCF File line: 800000
reading VCF File line: 900000
reading VCF File line: 1000000
data frame loaded



In [ ]:
# Load main vcf + append additional fields
df = aVCF.loadVaraintsFromVCF(VCF_PATH,VCF_FILE_NAME,SAMPLE_NAMES,True,bed)
df['chromPos']=df['chrom']+':'+df['pos']
df['Truth']=df.chromPos.isin(dfTruth['chromPos']) 
df['numCallers']= df['vennSegment'].apply(lambda x: x.count('-')+1) ## PP - dodgy.  need to fix
df['numCallers'][df['vennSegment'] == 'Intersection'] = 4 # PP - should not hardcode 4

reading vcf file. . .

reading VCF File line: 100000


<h3> High Level Stats

In [ ]:
df[['caller','variantType','pos']].groupby(['variantType','caller']).agg('count')

<h3> Overlapping Analysis

In [ ]:
sorteddf = df.sort(['chromFrac'], ascending=1)
sorteddf['nextRef'] = sorteddf['ref'].shift(-2)
sorteddf['nextPos'] = sorteddf['pos'].shift(-2)
sorteddf['nextChrom'] = sorteddf['chrom'].shift(-2)
sorteddf['nextVT'] = sorteddf['variantType'].shift(-2)
sorteddf['nextAlleleTumor1'] = sorteddf['alleleTumor1'].shift(-2)
sorteddf['nextAlleleTumor2'] = sorteddf['alleleTumor2'].shift(-2)
sorteddf['nextVennSegment'] = sorteddf['vennSegment'].shift(-2)
sorteddf['lengthRef'] = sorteddf['ref'].str.len()
sorteddf['Dist2Next'] = -sorteddf['pos'].astype(int).diff(-2)

overlapdf = sorteddf[(sorteddf['lengthRef']+0>sorteddf['Dist2Next']) & (sorteddf['Dist2Next']>0)]
overlapdf.head(20)

In [ ]:
filtereddf = overlapdf[(~overlapdf.vennSegment.isin(['freebayes','varscan','strelka1'])) \
                       & (overlapdf.vennSegment.str.contains('varscan')) \
                       #& (~overlapdf.nextVennSegment.str.contains('varscan')) \
                       & (overlapdf.variantType == 'INDEL') \
                       #& (overlapdf.variantSubType == 'DELETE')
                       & (overlapdf.variantType == overlapdf.nextVT)]
filtereddf[['nextVennSegment','vennSegment','variantType','nextVT','Dist2Next','lengthRef','ref','alleleTumor2', \
            'nextRef','nextAlleleTumor2','variantSubType','chrom','pos']].count()

<h3> SNP Venn

In [ ]:
snpdf = df[(df.variantType != 'SNP')]

In [ ]:
pd.pivot_table(snpdf, values='pos', index=['vennSegment'], columns=['Truth','caller'], aggfunc='count')

In [ ]:
vn.venn([snpdf[snpdf.caller == 'mutect']['chromPos'], \
         snpdf[snpdf.caller == 'strelka']['chromPos'], \
        snpdf[snpdf.caller == 'freebayes']['chromPos'], \
        snpdf[snpdf.caller == 'varscan']['chromPos'] \
        ],['mutect','strelka','freebayes','varscan'],figsize=(9,9))

### <hd1> Indel Venn

In [ ]:
indeldf= df[(df.variantType == 'INDEL')]

In [ ]:
pd.pivot_table(snpdf, values='pos', index=['vennSegment'], columns=['Truth','caller'], aggfunc='count')

In [ ]:
vn.venn([indeldf[indeldf.caller == 'strelka']['chromPos'], \
        indeldf[indeldf.caller == 'freebayes']['chromPos'], \
        indeldf[indeldf.caller == 'varscan']['chromPos'] \
        ],['strelka','freebayes','varscan'],figsize=(15,10))

<h3> Allelic Depth

In [ ]:
# BY CALLER
callers = df[df.caller != '70-30truth'].caller.unique()
for caller in callers:
    ser = df[df.caller == caller].allelicFreq
    ser = ser.sort_values()
    ser[len(ser)] = ser.iloc[-1]
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=caller+": median="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=" Allelic Frequency by Caller",figsize=[15,6])

In [ ]:
# BY CALLER BY VENN SEGMENT - Single Caller
myCaller = 'mutect'
vennSegments = df[df.caller == myCaller].vennSegment.unique()
for vennSegment in vennSegments:
    ser = df[df.vennSegment == vennSegment].allelicFreq
    ser = ser.sort_values()
    ser[len(ser)] = ser.iloc[-1]
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=vennSegment+": median="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=myCaller+" by Venn Segment",figsize=[15,6])

In [ ]:
# BY CALLER BY VENN SEGMENT - Single Caller FALSE POSITIVES
vennSegments = df[(df.caller == myCaller) & (df.Truth == False) ].vennSegment.unique()
for vennSegment in vennSegments:
    ser = df[(df.vennSegment == vennSegment) & (df.Truth == False)].allelicFreq
    ser = ser.sort_values()
    ser[len(ser)] = ser.iloc[-1]
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=vennSegment+": median="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=myCaller+" FALSE POSITIVES by Venn Segment",figsize=[15,6])

In [ ]:
# BY CALLER BY VENN SEGMENT - Single Caller TRUE POSITIVES
vennSegments = df[(df.caller == myCaller) & (df.Truth == True) ].vennSegment.unique()
for vennSegment in vennSegments:
    ser = df[(df.vennSegment == vennSegment) & (df.Truth == True)].allelicFreq
    ser = ser.sort_values()
    ser[len(ser)] = ser.iloc[-1]
    cum_dist = np.linspace(0.,1.,len(ser))
    ser_cdf = pd.Series(cum_dist, index=ser,name=vennSegment+": median="+str(round(ser.median(),2)))
    ser_cdf.plot(drawstyle='steps',legend=True,title=myCaller+" TRUE POSITIVES by Venn Segment",figsize=[15,6])

### Allelic Depth Scatter Plot

In [ ]:
# Combine segments
tempMutectDF = df[(df.caller =='mutect') & ((df.vennSegment == 'Intersection') \
            | ((df.vennSegment.str.contains('varscan')) & (df.vennSegment.str.contains('mutect'))))]
tempMutectDF = tempMutectDF.set_index(tempMutectDF['chromPos'])
tempvarscanDF = df[(df.caller =='varscan') & ((df.vennSegment == 'Intersection') \
            | ((df.vennSegment.str.contains('varscan')) & (df.vennSegment.str.contains('mutect'))))]
tempvarscanDF = tempvarscanDF.set_index(tempvarscanDF['chromPos'])
combinedDF = pd.merge(tempMutectDF, tempvarscanDF, on='chromPos', suffixes=['_mutect', '_varscan'])

In [ ]:
combinedDF.plot.scatter('allelicFreq_varscan','allelicFreq_mutect' \
                        ,figsize=[6,6] \
                        ,title="Comparison of Varscan and Mutect Allelic Frequency" \
                        ,xlim=[0,1],ylim=[0,1])

<h3> RAINBOW CHARTS

In [ ]:
## RAINBOW
#plt.scatter(df[df.caller == 'mutect'].chromFrac, df[df.caller == 'mutect'].allelicFreq, s=10, c='b', marker="s")